# Sampling pose DKF trained on H3.6M

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import re
import os
import addpaths
from load import loadDataset
import p2d_loader
import os
import numpy as np
from scipy.signal import convolve2d
import sys
del sys.argv[1:]

PFX = './chkpt-ikeadb/'
CONFIG_PATH = PFX + 'DKF_lr-8_0000e-04-vm-L-inf-structured-dh-50-ds-10-nl-relu-bs-20-ep-2000-rs-600-ttype-simple_gated-etype-mlp-previnp-False-ar-1_0000e+01-rv-5_0000e-02-nade-False-nt-5000-cond-True-ikeadb-acts-config.pkl'
WEIGHT_PATH = PFX + 'DKF_lr-8_0000e-04-vm-L-inf-structured-dh-50-ds-10-nl-relu-bs-20-ep-2000-rs-600-ttype-simple_gated-etype-mlp-previnp-False-ar-1_0000e+01-rv-5_0000e-02-nade-False-nt-5000-cond-True-ikeadb-acts-EP175-params.npz'
sys.argv.extend('-vm L -cond -infm structured -ds 10 -dh 50 -uid past-only'.split())

sys.argv.extend(['-reload', WEIGHT_PATH, '-params', CONFIG_PATH])

In [ ]:
dataset = loadDataset()

In [ ]:
from parse_args_dkf import params
from utils.misc import removeIfExists,createIfAbsent,mapPrint,saveHDF5,displayTime
from stinfmodel_fast.dkf import DKF
import stinfmodel_fast.learning as DKF_learn
import stinfmodel_fast.evaluate as DKF_evaluate

In [ ]:
act_names = dataset['p2d_action_names']
print('Action names: ' + ', '.join(map(str, act_names)))
one_hot_acts = {}
hot_vec_size = len(act_names)
for hot_bit, name in enumerate(act_names):
    one_hot_acts[name] = (np.arange(hot_vec_size) == hot_bit)
parents = dataset['p2d_parents']
print('Parents array: %s' % parents)

In [ ]:
use_cond = bool(params.get('use_cond', False))
params['savedir']+='-ikeadb'
# createIfAbsent(params['savedir'])

# Add dataset and NADE parameters to "params" which will become part of the
# model
for k in ['dim_observations','data_type']:
    params[k] = dataset[k]
mapPrint('Options: ',params)
if params['use_nade']:
    params['data_type']='real_nade'

# Remove from params
removeIfExists('./NOSUCHFILE')
reloadFile = params.pop('reloadFile')
pfile=params.pop('paramFile')
# paramFile is set inside the BaseClass in theanomodels
# to point to the pickle file containing params"""
assert os.path.exists(pfile),pfile+' not found. Need paramfile'
print 'Reloading trained model from : ',reloadFile
print 'Assuming ',pfile,' corresponds to model'
dkf  = DKF(params, paramFile = pfile, reloadFile = reloadFile)

In [ ]:
def smooth_seq(seq):
    assert seq.ndim in {2, 3}, seq.shape
    if seq.ndim == 3:
        rv = np.zeros_like(seq)
        for r in range(len(seq)):
            rv[r] = p2d_loader.gauss_filter(seq[r], sigma=1.0)
        return rv
    # 2d, filter whole thing
    return p2d_loader.gauss_filter(seq, sigma=1.0)

In [ ]:
# if not use_cond:
#     # No need to do conditional nonsense!
#     oodles_of_samples = dkf.sample(nsamples=50, T=1024)
#     sample_X, sample_Z = oodles_of_samples

#     print('Output shape: %s' % str(sample_X.shape))
#     mu = dataset['h36m_mean'].reshape((1, 1, -1))
#     sigma = dataset['h36m_std'].reshape((1, 1, -1))
#     real_X = insert_junk_entries(sample_X * sigma + mu)
#     dest_dir = './generated/'
#     try:
#         os.makedirs(dest_dir)
#     except OSError:
#         pass
#     for i, sampled_times in enumerate(real_X):
#         dest_fn = os.path.join(dest_dir, 'seq-%i.txt' % i)
#         print('Saving %s' % dest_fn)
#         np.savetxt(dest_fn, sampled_times, delimiter=',', fmt='%f')

#     # Do the same thing, but smoothed
#     smooth_dest_dir = './generated-smooth/'
#     try:
#         os.makedirs(smooth_dest_dir)
#     except OSError:
#         pass
#     for i, sampled_times in enumerate(real_X):
#         dest_fn = os.path.join(smooth_dest_dir, 'seq-%i.txt' % i)
#         print('Saving %s' % dest_fn)
#         smooth_times = smooth_seq(sampled_times)
#         np.savetxt(dest_fn, smooth_times, delimiter=',', fmt='%f')

In [ ]:
def scrape_by_act(poses, actions, one_hot_rep, count, sigma, mu, parents):
    # actions should be N*T*A
    assert np.prod(one_hot_rep.shape) == one_hot_rep.size, \
        "one-hot rep must be a vector"
    act_num = np.argmax(one_hot_rep.flatten())
    assert actions.ndim == 3, actions.shape
    act_nums = np.argmax(actions, axis=2)
    assert act_nums.shape == poses.shape[:2], \
        "mismatched action shape %s and pose shape %s" \
            % (actions.shape, poses.shape)
    # try to find sequences that feature part of the action
    has_act = np.nonzero((act_num == act_nums).any(axis=1))
    indices = has_act[np.random.permutation(len(has_act))][:count]
    if len(indices) < count:
        print('Only found %d instances of action with ID %d'
              % (len(indices), act_num))
    out_data = poses[indices]
    return p2d_loader.reconstruct_poses(out_data * sigma + mu, parents)

def sanitise_name(name):
    # for sanitising filenames
    return re.sub(r'[^a-z0-9_-]+', '-', name.lower()).strip('-')

In [ ]:
if use_cond:
    seqs_per_act = 9
    seq_length = 256
    dest_dir = './generated-wacts/'
    try:
        os.makedirs(dest_dir)
    except OSError:
        pass
        
    # start by generating some sequences for each action type
    for act_name, one_hot_rep in one_hot_acts.items():
        print('Working on action %s' % act_name)
        U = np.stack([one_hot_rep] * seq_length, axis=0)
        oodles_of_samples = dkf.sample(nsamples=seqs_per_act, T=seq_length, U=U)
        sample_X, sample_Z = oodles_of_samples
        mu = dataset['p2d_mean'].reshape((1, 1, -1))
        sigma = dataset['p2d_std'].reshape((1, 1, -1))
        real_X = p2d_loader.reconstruct_poses(sample_X * sigma + mu, parents)
        
        # Scrape some training poses, too
        train_poses = scrape_by_act(
            dataset['train'],
            dataset['train_cond_vals'],
            one_hot_rep,
            seqs_per_act,
            sigma,
            mu,
            parents)
        val_poses = scrape_by_act(
            dataset['valid'],
            dataset['val_cond_vals'],
            one_hot_rep,
            seqs_per_act,
            sigma,
            mu,
            parents)
        
        smooth_sampled_times = smooth_seq(
            real_X.reshape(real_X.shape[:2] + (16,))
        ).reshape(real_X.shape[:2] + (2, 8))
        actn = sanitise_name(act_name)
        dest_pfx = os.path.join(dest_dir, 'act-%s' % actn)
        dest_fn = dest_pfx + '.npz'
        print('Saving ' + dest_fn)
        np.savez(
            dest_fn, poses_gen=real_X,
            poses_smooth=smooth_sampled_times,
            poses_train=train_poses,
            poses_val=val_poses,
            parents=parents
        )

    # now choose random pairs of (distinct) actions and simulate
    # a transition at half-way point
    num_pairs = 10
    nacts = len(act_names)
    chosen_idxs = np.random.permutation(nacts * (nacts-1))[:num_pairs]
    act_pairs = [(act_names[idxp%nacts], act_names[idxp//nacts]) \
                 for idxp in chosen_idxs]
    for act1, act2 in act_pairs:
        print('Computing sequence for action %s -> %s' % (act1, act2))
        
        len1 = seq_length // 2
        len2 = seq_length - len1
        rep1 = one_hot_acts[act1]
        rep2 = one_hot_acts[act2]
        U = np.stack([rep1] * len1 + [rep2] * len2, axis=0)
        oodles_of_samples = dkf.sample(nsamples=seqs_per_act, T=seq_length, U=U)
        sample_X, sample_Z = oodles_of_samples
        mu = dataset['p2d_mean'].reshape((1, 1, -1))
        sigma = dataset['p2d_std'].reshape((1, 1, -1))
        real_X = p2d_loader.reconstruct_poses(sample_X * sigma + mu, parents)
        
        smooth_sampled_times = smooth_seq(
            real_X.reshape(real_X.shape[:2] + (16,))
        ).reshape(real_X.shape[:2] + (2, 8))
        act1n = sanitise_name(act1)
        act2n = sanitise_name(act2)
        dest_pfx = os.path.join(
            dest_dir,
            'trans-%s-to-%s' % (act1n, act2n))
        dest_fn = dest_pfx + '.npz'
        print('Saving ' + dest_fn)
        np.savez(
            dest_fn,
            poses_trans=sampled_times,
            poses_trans_smooth=smooth_sampled_times,
            parents=parents)